# **IP1_Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))


### **Generate Mathematical Model**

In [ ]:
problem = """ We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better.

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions.

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:
- n = Number of patients which is 17
- p = Number of proton fractions
- C = Maximum capacity of proton fractions which is 100
- BEDi(j, 15-j) = biological equivalent dose for patient i, with j proton fractions and 15-j photon fractions

Decision Variables:

Let xij = Number of proton fractions administered to patient i, ranging from 0 to 15 

Objective Function:

Maximize Σ(i=1 to 17) [BEDi(xi,15-xi)]

This implies that we are trying to maximize the total BED for all 17 patients by adjusting the number of proton fractions given to each patient.

Constraints:

1. Proton fractions limit: Σ(i=1 to 17) xi <= C 
(Capacity constraint, the total proton fractions must not exceed the capacity C)

2. Fraction limit for each patient: For each i (1 to 17), 0 <= xi <= 15 
(Each patient can have a maximum of 15 fractions which could be proton or photon or a mix)

3. Integer constraint: For each i (1 to 17), xi is an integer 
(This is because the number of fractions cannot be in decimals 

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response2.choices[0].message.content)

To implement this optimization problem using Pyomo, we first need to install pyomo library to python. You can do this via pip:

```python
pip install pyomo
```

Then, we can begin writing the Pyomo code for the problem:

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define sets
P = range(17) # Set of patients

# Sample data for BEDi
BEDi = {(i,j):1 for i in P for j in range(16)}  # Here we assume that BEDi(xi,15-xi) is always 1

# Define variables
model.x = Var(P, range(16), domain=NonNegativeIntegers)

# Define objective function
model.obj = Objective(expr = sum(BEDi[i, j]*model.x[i,j] for i in P for j in range(16)), sense=maximize)

# Define constraints
model.Constraint1 = Constraint(expr = sum(model.x[i,j] for i in P for j in range(16)) <= 100) # Proton fractions limit

# Fraction limit for each patient
for i in P:
    model.add_component("Constraint2_%s"%i, Constraint(expr = sum(model.x[i,j] for j in range(16)) <= 15))

# Define and solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in P:
    for j in range(16):
        print(f"Number of proton fractions given to patient {i+1} with j protons: {model.x[i,j].value}")
print(f"Objective function (total BED): {model.obj()}")
```

In the above code, we used sample data for BEDi by assuming that BEDi(xi,15-xi) is always 1, which means each fraction of proton or photon treatment gives a BED of 1 to each patient. 

Please replace the sample data with actual data to solve the problem correctly for actual scenarios.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define sets
P = range(17) # Set of patients

# Sample data for BEDi
BEDi = {(i,j):1 for i in P for j in range(16)}  # Here we assume that BEDi(xi,15-xi) is always 1

# Define variables
model.x = Var(P, range(16), domain=NonNegativeIntegers)

# Define objective function
model.obj = Objective(expr = sum(BEDi[i, j]*model.x[i,j] for i in P for j in range(16)), sense=maximize)

# Define constraints
model.Constraint1 = Constraint(expr = sum(model.x[i,j] for i in P for j in range(16)) <= 100) # Proton fractions limit

# Fraction limit for each patient
for i in P:
    model.add_component("Constraint2_%s"%i, Constraint(expr = sum(model.x[i,j] for j in range(16)) <= 15))

# Define and solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in P:
    for j in range(16):
        print(f"Number of proton fractions given to patient {i+1} with j protons: {model.x[i,j].value}")
print(f"Objective function (total BED): {model.obj()}")

Number of proton fractions given to patient 1 with j protons: 15.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of

In [57]:
from pyomo.environ import *
import pandas as pd
# Create a concrete model
model = ConcreteModel()

# Define sets
P = range(16) # Set of patients

# START - human inputted data
df = pd.read_csv('ProblemData (2).csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i, j)] = dose

#END
# Define variables
model.x = Var(P, range(16), domain=NonNegativeIntegers)

# Define objective function
model.obj = Objective(expr = sum(BED[i,j]*model.x[i,j] for i in P for j in range(16)), sense=maximize)

# Define constraints
model.Constraint1 = Constraint(expr = sum(model.x[i,j] for i in P for j in range(16)) <= 100) # Proton fractions limit

# Fraction limit for each patient
for i in P:
    model.add_component("Constraint2_%s"%i, Constraint(expr = sum(model.x[i,j] for j in range(16)) <= 15))

# Define and solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in P:
    for j in range(16):
        print(f"Number of proton fractions given to patient {i+1} with j protons: {model.x[i,j].value}")
print(f"Objective function (total BED): {model.obj()}")

Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of 

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [58]:
from pyomo.environ import *
import pandas as pd
# Create a concrete model
model = ConcreteModel()

# Define sets
P = range(16) # Set of patients

# START - human inputted data
df = pd.read_csv('ProblemData (2).csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i, j)] = dose

#END
# Define variables
model.x = Var(P, range(16), domain=NonNegativeIntegers)

# Define objective function
model.obj = Objective(expr = sum(BED[i,j] for i in P for j in range(16)), sense=maximize)

# Define constraints
model.Constraint1 = Constraint(expr = sum(model.x[i,j] for i in P for j in range(16)) <= 100) # Proton fractions limit

# Fraction limit for each patient
for i in P:
    model.add_component("Constraint2_%s"%i, Constraint(expr = sum(model.x[i,j] for j in range(16)) <= 15))

# Define and solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in P:
    for j in range(16):
        print(f"Number of proton fractions given to patient {i+1} with j protons: {model.x[i,j].value}")
print(f"Objective function (total BED): {model.obj()}")

Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of proton fractions given to patient 1 with j protons: 0.0
Number of 